# Machine learning: Final Project

## Authors: Maximilian Janisch and Marco Bertenghi

### Problem Description

We have been assigned a multi-class classification problem, that is we are interested in correctly predicting if a given input in $\mathcal{X} = \mathbb{R}^d$ belongs to one of __three__ classes (i.e. $\mathcal{Y} = \{0,1,2\}$). We are instructed to solve this multi-class classification problem by reducing it to a sequence of binary classification problems (i.e. where $\mathcal{Y}'=\{0,1\}$). We are __not__ allowed to use the multi-class implementation of sklearn and one of the models we must implement must be the random forest model. The goal of the project is then to describe and document in detail how to find and implement the best machine learning algorithm for the dataset assigned to us.


We are given a __E-SCOOTER DATASET__. The dataset contains data about the profitability of e-scooter companies. The goal is to predict, as well as possible, the profitability of e-scooter companies by using the information provided. The dataset comes as a csv file (comma-separated values file), named dataset20.csv and contains the following description:

__E-SCOOTER DATASET__

----------------------------------------------

__Dataset description__:
The data is compiled from cities, towns, and small villages in Germany, Austria, and Switzerland, classified according to the profitability of e-scooter companies active in that location. There are __thirteen different features__ associated with each location. The goal is to predict the profitability of an e-scooter company (feature "class") from the other features.

__Attention!__
Please notice that the data has been artificially generated. The dataset does not reflect real-world statistical correlations between features and labels.

	Number of samples: 500
	Number of features: 13 (numeric and strings) + one column of class labels (0,1,2)
	Features description:
		pub_trans: public transport index
		price: price to rent per km
		temperature: average tempereture during summer
		inhabitants: number of inhabitants
		registered: number of registered users in thousand
		country: country
		id: internal dataset code
		nr_counterparts: nr_counterparts
		cars: number of cars per inhabitant
		labour_cost: average labour cost in thousand per month
		humidity: absolute humidity in g/m3
		windspeed: average wind speed in km/h
		size: size of city center in km2
		class: profitability (0 = loss, 1 = balanced, 2 = profit) <--- LABEL TO PREDICT

----------------------------------------


We notice that:

+ We have a total of 500 samples.
+ We have a total of 13 features.
 + Not all features will be relevant, more on that later.
+ The profitability is a class, it is the label we want to predict.
+ The CSV file uses ';' as a delimiter.

# Coding part

## Reading in the data

In [1]:
# Preliminary code, libraries we want to load
import os
import csv

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder  # Todo: der OneHotEncoder wird im Moment nicht verwendet
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis  # im Moment nicht verwendet

from RandomForestImplementation.randomForest import RandomForestClassifier
from RandomForestImplementation.multiclass import OneVsOneClassifier

In [2]:
# Preliminary code, global variables
DATA_FILE = os.path.join("data", "dataset20.csv")
CLASSES = ["0 = loss", "1 = balanced", "2 = profit"]

We read in the data as follows:

In [3]:
with open(DATA_FILE, 'r') as f:
    data = csv.reader(f, delimiter = ";")

    row = data.__next__()
    rows = [row for row in data]
    features_names = np.array(row)
    CATEGORICAL_COLUMNS = [5, 6, 7]  # these are the columns "country", "id" and "nr_counterparts" which don't contain numerical data
    
    x = np.array([row[0:13] for row in rows])  # we put all the features in the x vector
    y = np.array([row[13] for row in rows]).astype(int)  # the labels go to the y vector

We will convert the categorical values to ordinals. This is a bit dangerous because the ordinals have a natural ordering which provides incorrect information about our classifications. However, it doesn't make a difference for the plots or the f-tests.

In [4]:
x = np.where(x == "", np.nan, x)  # This replaces all missing data (imported as "") by numpy NaNs. We will get back to missing data during data preprocessing
ordinalEnc = OrdinalEncoder()
x[:, CATEGORICAL_COLUMNS] = ordinalEnc.fit_transform(x[:, CATEGORICAL_COLUMNS])  # ordinal encoding of the categorical columns
x = x.astype(float)

As a brief sanity check: We can have a look at the feature names and the first representative in x.

In [5]:
print("We have the following features: \n", features_names)
print("--------------------------------")
print("The first row looks as follows: \n", x[0,:])
print("--------------------------------")
print("The first row has class label :\n", y[0])

We have the following features: 
 ['pub_trans' 'price' 'temperature' 'inhabitants' 'registered' 'country'
 'id' 'nr_counterparts' 'cars' 'labour_cost' 'humidity' 'windspeed' 'size'
 'class']
--------------------------------
The first row looks as follows: 
 [1.42026351e+01 1.90186463e-02 2.51751307e+01 2.35528166e+00
 1.18021208e+02 1.00000000e+00 4.25000000e+02 0.00000000e+00
 4.13613819e-01 2.23781367e+00 2.66531063e+00 2.83177019e+00
 7.48581880e-01]
--------------------------------
The first row has class label :
 2


Here is the shape of our data:

In [6]:
print("Shape of the features: \n", x.shape)  # Sanity check, to see what our data looks like.
print("Shape of the labels: \n", y.shape)

Shape of the features: 
 (499, 13)
Shape of the labels: 
 (499,)


# Data preprocessing

#### Feature selection

We are given a varities of features as we've already seen in the problem description. We first want to investigate which features are _relevant_ for our purposes. One suitable way to do this is with the help of data-visualisation, which allows us to draw conclusions about the relevancy of each and every feature (numeric or not).

_Example: The feature 'id' stands for the internal dataset code. We consider this feature to be irrelevant as the success of a company will not depend on what they label their products internally. Similarly, the number of counter parts will not play a role when it comes to the success of a company, in fact the customer will not know how many counter parts their product is made of. Hence we will 'forget' about these features._

#### Data imputation

We shall first investigate how _complete_ our data is, i.e. we want to see if there are any 'NaN' (Not a Number) values present.

In [7]:
for i in range(x.shape[1]):
    if np.isnan(x[:, i]).any():
        print("Feature", i, "called", f'"{features_names[i]}"', "has", np.isnan(x[:,i]).sum(), "NaN value(s)")

Feature 3 called "inhabitants" has 4 NaN value(s)
Feature 12 called "size" has 3 NaN value(s)


The code shows that the features 'inhabitants' and 'size' have NaN values present. We can deal with missing data in the following ways:
+ Remove the datapoint (but that is not a good idea here since our data set is already small, hence we don't have the luxury removing even more data from our dataset)
+ **Data imputation** techniques, i.e. substitute the NaN with a plausible value: 
    + the mean over non-NaN data of same feature, 
    + the mid-point of the range of non-NaN data range for same feature, 
    + the prediction of a regression problem run on the remaining features to predict missing feature.

We choose the simplest solution and replace the missing values with the mean value of the feature:

In [8]:
## Fixing NaN values in inhabitants feature (index 3) and size feature (index 12)
for column_index in range(x.shape[1]):
    x[:, column_index] = np.where(np.isnan(x[:, column_index]), np.nanmean(x[:, column_index]), x[:, column_index])

We shall now produce and analyse various scatter plots and use them in order to determine whether a feature is relevant or not.

In [ ]:
def scatterplot_kth_column(k, colors=["red", "green", "blue"]):
    scatter = plt.scatter(range(1, 500), x[:, k], c=y, cmap=ListedColormap(colors))
    plt.ylabel(features_names[k])
    plt.legend(handles=scatter.legend_elements()[0], labels=CLASSES)

    return scatter

scatterplot_kth_column(0)

We observe a clustering of the data with respect to its classes, which we color-coded by RBG (see legend). We conclude that the _pub_trans_ feature is relevant.

In [ ]:
scatterplot_kth_column(1)

We observe a clustering of the data with respect to its classes, which we color-coded by RBG. We conclude that the _price_ feature is relevant. This also makes intuitive sense, that the price will play a role in the success of the companies sales.

_Note: Since the data is artificially generated, of course a negative price makes no sense._

In [ ]:
scatterplot_kth_column(2)

We observe a clustering of the data with respect to its classes, which we color-coded by RBG. We conclude that the _temperature_ feature is relevant. This also makes intuitive sense, that the temperature might determine how often people are outside and hence use an E-Scooter, which translates to a companies success in sales. For example: One might not want to start an E-Scooter company in Siberia.

_Note: The data is artificial, the most loss seems to be made at around 20-22.5 degrees Celsius. Fair and rather cold weather (i.e. below 17.5 degrees Celsius) still makes balanced profit._

In [ ]:
scatterplot_kth_column(9)

We observe a clustering of the data with respect to its classes, which we color-coded by RBG. We conclude that the _labour_cost_ feature is relevant. This also makes intuitive sense, that the labour cost plays a role in the companies success. If the labour costs are low, the company is more likely to make a profit than with high labour costs, this is also reflected in the graph above.

In [ ]:
fig = plt.figure(figsize=(15, 15))

for k, feature in enumerate([3, 4, 5, 6, 7, 8, 10, 11, 12]):  # scatterplots for some other features
    plt.subplot(3, 3, k+1)
    scatterplot_kth_column(feature)

The plots above show a much more _chaotic_ behaviour of the data. There is no apparent relationship between the classes and the features. For now we shall consider them as _rather irrelevant_.

We still need to discuss whether the feature 'country' is relevant or not. We shall do this by the means of counting:

In [14]:
def counting(number):
    """
    A naive counting function which helps us determine whether a given country is successful in sales or not. 
    It does so by counting the instances of loss, balanced and profit classes for each country.
    Usage: 0 = Austria, 1 = Germany, 2 = Switzerland.
    """
    
    country_name = ordinalEnc.inverse_transform([[number,0,0]])[0,0]  # the two zeros after number are not important, they will give the first id and first number of counterparts
    
    count = np.bincount(y[x[:, 5] == number])
    
    print(country_name, "has:", count[0], "instances of loss", count[1], "instances of balanced and", count[2], "instances of profit")

for i in (0,1,2): 
    counting(i)

Austria has: 59 instances of loss 62 instances of balanced and 49 instances of profit
Germany has: 54 instances of loss 56 instances of balanced and 57 instances of profit
Switzerland has: 55 instances of loss 58 instances of balanced and 49 instances of profit


Our analysis suggest that the country feature is also irrelevant as the instances of loss/balance/profit is evenly spread among the three countries. This makes intuitive sense: If a E-Scooter company is successful in one of the three neighbour countries Austria, Germany, Switzerland then it should also be successful in the other two. 

On top of the visual (statistical) analysis we've carried out so far, we shall incorporate an _f-test_ in order to further see which of the features are relevant or not:

In [15]:
selector = SelectKBest(score_func=f_classif, k="all")
selection = selector.fit(x, y)
print(f"{'Feature name':<17}Score")
for feature_name, score in zip(features_names, selection.scores_):
    print(f"{feature_name:<17}{score}")

Feature name     Score
pub_trans        838.7327211156739
price            507.97390601771116
temperature      741.9405923087922
inhabitants      0.3797211844807062
registered       0.9311583750278856
country          0.043267894972736455
id               0.9733206062013445
nr_counterparts  0.48818426449611024
cars             0.23499639468285957
labour_cost      380.0286669426751
humidity         2.4577930386538798
windspeed        2.883947910173691
size             1.2846730121640408


In conjunction with our visual statistical analysis the _f-test_ confirms our hypothesis of relevant respectively irrelevant features.

Evidently, not all features are relevant and we might as well _'forget'_ about them. We recall our features and mark them as relevant respectively irrelevant according to our observations above:

		pub_trans: public transport index  (relevant)
		price: price to rent per km (relevant)
		temperature: average tempereture during summer (relevant)
		inhabitants: number of inhabitants (irrelevant)
		registered: number of registered users in thousand (irrelevant)
		country: country (irrelevant)
		id: internal dataset code (irrelevant)
		nr_counterparts: nr_counterparts (irrelevant)
		cars: number of cars per inhabitant (irrelevant)
		labour_cost: average labour cost in thousand per month (relevant)
		humidity: absolute humidity in g/m3 (relevant) [maybe only a slight relevance]
		windspeed: average wind speed in km/h (relevant) [maybe only a slight relevance]
		size: size of city center in km2 (irrelevant)
		class: profitability (0 = loss, 1 = balanced, 2 = profit) <--- LABEL TO PREDICT !!!

In accordance with our observations, we limit ourselves to the following features:

In [16]:
CHOSEN_FEATURES = [0,1,2,9,10,11]
X = x[:, CHOSEN_FEATURES]
features_names = features_names[CHOSEN_FEATURES]

In [17]:
print("New features: \n", features_names)
print("The first row now looks like this: \n", x[0,:])

New features: 
 ['pub_trans' 'price' 'temperature' 'labour_cost' 'humidity' 'windspeed']
The first row now looks like this: 
 [1.42026351e+01 1.90186463e-02 2.51751307e+01 2.35528166e+00
 1.18021208e+02 1.00000000e+00 4.25000000e+02 0.00000000e+00
 4.13613819e-01 2.23781367e+00 2.66531063e+00 2.83177019e+00
 7.48581880e-01]


## Shuffling and data train/test splitting

### Shuffling

We know that usually datapoints in a sample are not realizations of a perfectly _i.i.d. sequence of random variables_. Instead, the sample might be generated in such a way that the datapoints have some degree of time dependence without us noticing, or they might be correlated in some other way among each other. Generally speaking, we might not want to _trust_ the data as it is distributed to us, since we do not know in what way it was gathered (dependencies and so on).

Moreover, some algorithms may depend on the order of the datapoints in the sample.

In order to reduce these efffects as much as possible,it is always a good idea to __shuffle__ the datapoints before usage.

### Train, test and validation splitting

To obtain a result of how good our model will be we cannot plainly use the empirical error obtained on the training dataset. This error is biased as we trained the model in such a way as to minimize it!

Instead, one typically would like to compute the empirical error on new, previously __unseen__ data to obtain a better estimate of how good the model describes reality. These "new, previously unseen" data __must be set aside from the beginning and separated from the training dataset__ into a test dataset.

The test dataset __will not be used for training__ and we will __use it only at the end__, in order to come up with an estimate of how good our model is.

The process of evaluating the prediction error of our model using the empirical error on the test dataset is called testing. As a rule of thumb around 20% of your total data should be hold back for testing in the end.

Furthermore we want to use some of the training data as a validation set, over which one can evalutate the success of the algorithm’s output predictor. This procedure is called **validation**. In this case, we separate our data set into three distinct sets of labeled examples:

1. training set,
2. validation set,
3. test set.

There’s no optimal proportion to split the dataset into these three subsets. The rule of thumb is to use 70% of the dataset for training, 15% for validation and 15% for testing. The validation set is typically used to optimize hyperparameters.

Using Scikit learn, the notion of train/test/validation splitting and shuffling can be implemented as follows:

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, shuffle=True) # direct shuffle and split with sklearn
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=1/2, shuffle=True)
print("The X and y training sets have the shape: \n", X_train.shape, y_train.shape)
print("The X and y testing sets have the shape: \n", X_test.shape, y_test.shape)
print("The X and y validation sets have the shape: \n", X_val.shape, y_val.shape)

The X and y training sets have the shape: 
 (332, 6) (332,)
The X and y testing sets have the shape: 
 (83, 6) (83,)
The X and y validation sets have the shape: 
 (84, 6) (84,)


## Model implementations

One of the models we must implement (according to the project description) is the random forest model.

Our implementation of the RandomForest algorithm already natively supports multiple labels:

In [19]:
forest_multi_classif = RandomForestClassifier(n_estimators=10, random_state=1)
forest_multi_classif.fit(X_train, y_train)
forest_multi_pred = forest_multi_classif.predict(X_test)
print("Accuracy:", (forest_multi_pred==y_test).sum() / len(y_test))

Accuracy: 0.8795180722891566


We have also implemented a class *OneVsOneClassifier* which makes it easy to perform OneVsOne classification with any classification estimator:

In [21]:
forest_classif = OneVsOneClassifier(RandomForestClassifier, n_classes=3, n_estimators=10, random_state=1)
forest_classif.fit(X_train, y_train)
forest_pred = forest_classif.predict(X_test)
print("Accuracy:", (forest_pred==y_test).sum() / len(y_test))

Accuracy: 0.9036144578313253


##### Todo: Text anpassen
We observe that we get the same percentage of accuracy. The result is also pretty good considering the data is unseen.

Next we shall implement some machine learning algorithms provided by scikit-learn. Note that since we are not allowed to use a multiclass classification approach directly (except for the random forest) we must first turn our multiclass classification problem into a binary classification problem. To this extend we make use of the OvO (One vs One) classifier method.


In [22]:
MLP_classif = OneVsOneClassifier(MLPClassifier, n_classes=3, learning_rate="invscaling")
MLP_classif.fit(X_train, y_train)
MLP_pred = MLP_classif.predict(X_test)
print("Accuracy:", (MLP_pred==y_test).sum()/len(y_test))

c:\users\maximilian\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
c:\users\maximilian\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.8674698795180723


c:\users\maximilian\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In the exercise classes (class 10) we've seen the $k$-nearnest neighbour algorithm. The $k$-nearest neighbour algorithm is arguably one of the most simple classification algorithm existing. The general idea is to predict the label of a new datapoint according to the label(s) of the closest training point(s) to it. Intuitively, it is a good idea to simply decide the label of a newly observed datapoint according to the label of the closest given point in the training set. Considering only one neighbour would lead to some evident problems. The role of $k$ is that more neighbours means smoother boundary and smoother decision boundary corresponds to simpler model. On the other hand, the boundary for small values of $k$ gets very rough. Hence, the decision rule gets more complicated. Put differently, low k corresponds to high model complexity and high $k$ corresponds to low model complexity.

In [23]:
neighbors_classif = OneVsOneClassifier(KNeighborsClassifier, n_classes=3, random_state=1)
neighbors_classif.fit(X_train, y_train)
neighbors_pred = neighbors_classif.predict(X_test)
print("Accuracy:", (neighbors_pred==y_test).sum() / len(y_test))

Accuracy: 0.891566265060241


Another algorithm we have seen in the exercise classes (class 8) is the support vector machine (SVM).

In [24]:
svc_classif = SVC()  # Note that the support vector machine is by default a OneVsOne algorithm
svc_classif.fit(X_train, y_train)
svc_pred = svc_classif.predict(X_test)
print("Accuracy:", (svc_pred==y_test).sum() / len(y_test))

Accuracy: 0.8795180722891566


In [25]:
adaboost_classif = OneVsOneClassifier(AdaBoostClassifier, n_classes=3, random_state=1)
adaboost_classif.fit(X_train, y_train)
adaboost_pred = adaboost_classif.predict(X_test)
print("Accuracy:", (adaboost_pred==y_test).sum()/len(y_test))

Accuracy: 0.8795180722891566


In [26]:
bagging_classif = OneVsOneClassifier(BaggingClassifier, n_classes=3, random_state=1)
bagging_classif.fit(X_train, y_train)
bagging_pred = bagging_classif.predict(X_test)
print("Accuracy:", (bagging_pred==y_test).sum()/len(y_test))

Accuracy: 0.8795180722891566


In [27]:
gradient_classif = SGDClassifier(random_state=1)  # note that the SGDClassifier is by default a OneVsRest algorithm
gradient_classif.fit(X_train, y_train)
gradient_pred = gradient_classif.predict(X_test)
print("Accuracy:", (gradient_pred==y_test).sum()/len(y_test))

Accuracy: 0.6265060240963856


In [28]:
gaussian_classif = OneVsOneClassifier(GaussianProcessClassifier, n_classes=3, random_state=1)
gaussian_classif.fit(X_train, y_train)
gaussian_pred = gaussian_classif.predict(X_test)
print("Accuracy:", (gaussian_pred==y_test).sum()/len(y_test))

Accuracy: 0.891566265060241
